In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from copy import deepcopy
from torch.nn.utils.rnn import pad_sequence
from utils.reparam_module import ReparamModule
import numpy as np
import random
from tqdm import tqdm

In [5]:
dataset_name = 'beauty'
full_dataset_name = 'amazon-beauty'
num_item_dict = {
    'toy': 11925,
    'sport': 18358,
    'beauty':12102,
}
path = f'./{dataset_name}-pair.pth'
data = torch.load(path)
num_item = num_item_dict[dataset_name]
SOS = num_item
EOS = num_item + 1

In [6]:
interval = list(range(0, 25001, 5000))
rst_list = []
for i in range(len(interval) - 1):
    rst_list.append(torch.load(f'f-seq-con-{dataset_name}-{interval[i]}-{interval[i + 1]}.pth'))
rst = []
for _ in rst_list:
    rst += _

In [7]:
max_seq_len = 50
def truncate_or_pad(seq):
    cur_seq_len = len(seq)
    if cur_seq_len > max_seq_len:
        return seq[-max_seq_len:]
    else:
        return seq + [0] * (max_seq_len - cur_seq_len)


train_set = set()

for pattern in rst:
    seq = pattern.tolist()[1:-1]
    seq_len = len(seq)
    for _ in range(1, seq_len):
        train_set.add(tuple(
            truncate_or_pad(seq[:_]) + [seq[_]],
        ))

train_list = []
for _ in train_set:
    train_list.append([
        1,
        list(_)[:-1],
        list(_)[-1],
        sum([a != 0 for a in list(_)[:-1]]),
        1,
        0
    ])
print(len(train_list))

126442


In [ ]:
ori_pattern = torch.load(f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_new-pure.pth')


In [ ]:
torch.save(train_list + ori_pattern, f'./dataset/{full_dataset_name}-noise-50/{dataset_name}/train_gene.pth')